In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import pandas as pd

class ACOOptimize(object):
    def __init__(self, no_cities, data):
        self.ant_count = 500  # number of ants
        self.alpha = 1  # pheromone importance factor
        self.beta = 5#5  # heuristic factor function
        self.rho = 0.65#0.65#  0.1  # pheromone volatile factor
        self.Q = 100  # constant coffecieint
        self.no_cities = no_cities  # city ​​size
        self.coordinate = data  # city coordinates
        self.mat_T = np.zeros([no_cities, no_cities])  # Pheromone Matrix
        self.record = [[0 for _ in range(no_cities)] for _ in range(self.ant_count)]  # generated ant colony
        self.iter = 1
        self.iter_max = 50 
        self.dis_mat = self.comp_dis_matrix(no_cities, self.coordinate)  # distance bn cities
        self.Eta = 10. / self.dis_mat  # heuristic function
        self.routes = None  # length of each individual in ant colony
        # Store the final route at each temperature and draw the convergence graph
        self.loop_x = []
        self.loop_y = []

    def greedy_initialize(self, dis_mat, no_total, no_cities):
        start_idx = 0
        result = []
        for i in range(no_total):
            rest = [i for i in range(0, no_cities)]
            # All starting points have been generated
            if start_idx >= no_cities:
                start_idx = np.random.randint(0, no_cities)
                result.append(result[start_idx].copy())
                continue
            curr = start_idx
            rest.remove(curr)
            # find a nearest neighbor route
            result_one = [curr]
            while len(rest) != 0:
                min_dist = math.inf
                choose_dist = -1
                for j in rest:
                    if dis_mat[curr][j] < min_dist:
                        min_dist = dis_mat[curr][j]
                        choose_dist = j

                curr = choose_dist
                result_one.append(choose_dist)
                rest.remove(choose_dist)
            result.append(result_one)
            start_idx += 1
        routelens = self.comp_route(result)
        sortindex = np.argsort(routelens)
        index = sortindex[0]
        result = result[index]
        for x in range(len(result)-1):
            s = result[x]
            s2 = result[x+1]
            self.mat_T[s][s2]=1
        self.mat_T[result[-1]][result[0]] = 1
    

    # roulette selection

    def random_choose(self, c):
        random.seed(2023)
        y = np.random.rand()
        for j, k in enumerate(c):
            y -= k
            if y <= 0:
                break
        return j

    # generate ant colony

    def generate_ants(self, no_cities):
        for i in range(self.ant_count):
            origin = np.random.randint(no_cities - 1)
            self.record[i][0] = origin
            novisit = list([y for y in range(no_cities) if y != origin])
            curr = origin
            j = 1
            while len(novisit) != 0:
                AP = []
                # Calculation of transition probabilities between cities by pheromones
                for v in novisit:
                    AP.append(self.mat_T[curr][v] ** self.alpha * self.Eta[curr][v] ** self.beta)
                sum_P = sum(AP)
                AP = [y / sum_P for y in AP]
                # Roulette to choose a city
                index = self.random_choose(AP)
                curr = novisit[index]
                self.record[i][j] = curr
                novisit.remove(curr)
                j += 1

    # Calculate the distance between different cities
    def comp_dis_matrix(self, no_cities, coordinate):
        dis_mat = np.zeros((no_cities, no_cities))
        for i in range(no_cities):
            for j in range(no_cities):
                if i == j:
                    dis_mat[i][j] = np.inf
                    continue
                c = coordinate[i]
                d = coordinate[j]
                dist = np.sqrt(sum([(y[0] - y[1]) ** 2 for y in zip(c, d)]))
                dis_mat[i][j] = dist
        return dis_mat

    # Calculate the length of a route
    def comp_routelength(self, route, dis_mat):
        c = route[0]
        d = route[-1]
        result = dis_mat[c][d]
        for i in range(len(route) - 1):
            c = route[i]
            d = route[i + 1]
            result += dis_mat[c][d]
        return result
    # Calculate the length of a group
    def comp_route(self, routes):
        result = []
        mean=[]
        median=[]
        for one in routes:
            length = self.comp_routelength(one, self.dis_mat)
            result.append(length)

        return result,np.mean(result),np.median(result)

    # update pheromone
    def update_pheromone(self):
        pheromone_update = np.zeros([self.no_cities, self.no_cities])
        routes,mean,median = self.comp_route(self.record)
        for i in range(self.ant_count):
            for j in range(self.no_cities - 1):
                a = self.record[i][j]
                b = self.record[i][j + 1]
                pheromone_update[a][b] = pheromone_update[a][b] + self.Q / routes[i]
            a = self.record[i][0]
            b = self.record[i][-1]
            pheromone_update[a][b] = pheromone_update[a][b] + self.Q / routes[i]
        self.mat_T = (1 - self.rho) * self.mat_T + pheromone_update

    def aco(self):
        best_length = math.inf
        best_route = None
        results_dict = {
            "generation": [],
            "best_fitness": [],
            "mean_fitness": [],
            "median_fitness": []}
        for cnt in range(self.iter_max):
            # spawn new ant colony
            self.generate_ants(self.no_cities)  
            self.routes,mean,median = self.comp_route(self.record)
            # Take the optimal solution of the ant colony
            dist_lenth = min(self.routes)
            dist_route = self.record[self.routes.index(dist_lenth)]

            # Visualize the initial route
            if cnt == 0:
                init_show = self.coordinate[dist_route]
                init_show = np.vstack([init_show, init_show[0]])
            # update optimal solution
            if dist_lenth < best_length:
                best_length = dist_lenth
                best_route = dist_route
            # update pheromone
            self.update_pheromone()

            # save result
            self.loop_x.append(cnt)
            self.loop_y.append(best_length)
            #print(cnt,best_length,mean,median)
            results_dict["generation"].append(cnt)
            results_dict["best_fitness"].append(best_length)
            results_dict["mean_fitness"].append(mean)
            results_dict["median_fitness"].append(median)
        return best_length, best_route,results_dict

    def run(self):
        best_length, best_route,results = self.aco()
        return self.coordinate[best_route], best_length,results


# read data

def read_tsp(directory):
    rows = open(directory, 'r').readlines()
    assert 'NODE_COORD_SECTION\n' in rows
    idx = rows.index('NODE_COORD_SECTION\n')
    tsp = rows[idx + 1:-1]
    dist = []
    for i in tsp:
        i = i.strip().split(' ')
        if i[0] == 'EOF':
            continue
        disti = []
        for x in i:
            if x == '':
                continue
            else:
                disti.append(float(x))
        if disti == []:
            continue
        dist.append(disti)
    tsp = dist
    return tsp


dataset = read_tsp('/content/st70.tsp')

dataset = np.array(dataset)
dataset = dataset[:, 1:]
# Add a line because it will go back to the beginning
show_data = np.vstack([dataset, dataset[0]])
all_results = []
for i in range(0,30):
 random.seed(i)
 print(i)
 aco = ACOOptimize(no_cities=dataset.shape[0], data=dataset.copy())
 Best_route, Best,results = aco.run()
 all_results.append(pd.DataFrame(results))
final_results = pd.concat(all_results, axis=1)
#print(final_results)

Best_route = np.vstack([Best_route, Best_route[0]])
plt.plot(Best_route[:, 0], Best_route[:, 1], marker='o', markerfacecolor='red')
plt.title('st70:ACO result')
plt.show()
 

In [ ]:
final= final_results.drop('generation', axis=1).reset_index()


In [ ]:
final_results.to_csv("file_aco_50iter_new_500pop.csv")

In [ ]:
import numpy as np

def parseData(data, firstcolumn, noRuns):
    col = firstcolumn
    
    allstats = (data.shape[1]-1)/noRuns   # how many stats were collected. Omit the first column (Generations)
    cols = np.arange(col, noRuns*allstats+1, allstats, dtype=int)
    #print(cols)
    subdata = data.iloc[:,cols]
    # print(subdata)
    noGens = data.shape[0]
    pdata = np.zeros((noGens, 4))
    for i in range(noGens):
        pdata[i,0] = i+1
        pdata[i,1] = np.mean(subdata.iloc[i,:].mean()) # mean(subdata[i,])
        pdata[i,2] = 1.96*np.std(subdata.iloc[i,:])/np.sqrt(noRuns) # 1.96*sd(rowMeans(subdata))/sqrt(noRuns) # compute the length of error bar. 
        pdata[i,3] = np.min(subdata.iloc[i,:]) # mean(subdata[i,])
    return pdata


In [ ]:
data = parseData(final, 1, 30)
mean_second_col = np.mean(data[:,1])
min_second_col = np.min(data[:,1])

print("mean ",mean_second_col)
print("min ",min_second_col)

mean  877.3637230241402
min  720.738385285575
